# 音楽音響信号の〇〇gram

音楽音響信号（i.e. 波形データ）に対する情報処理は、だいたい**時間周波数表現（time-frequency representation）** への変換から始まります。多種多様な音成分が混合された、極めて複雑な信号である音楽音響信号を扱う際、波形（waveform）という１次元データから音楽的な情報を直接見出すことは極めて困難です。波形を周波数成分に分解することで、分析したい情報に一歩近づくことができます。

本ページでは、音楽情報処理でよく使われる時間周波数表現（および関連する音響特徴量）とその特徴を紹介します。画像として可視化可能な２次元データであることから、これらの表現はだいたい「〇〇gram」と名付けられています。

## STFT Spectrogram

STFT（Short Time Fourier Transform、短時間フーリエ変換）スペクトログラムは、波形データから一定間隔に切り取った波形フレームをフーリエ変換し、時間軸上に並べることで作られます。
0Hz~ナイキスト周波数までの範囲を均等に分割した、各周波数域の成分の時間変化を知ることができます。

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display

waveform, sample_rate = librosa.load('./assets/example1.ogg', sr=22050)
waveform = waveform[:sample_rate*10]   # 最初の10秒
stft_spectrogram = librosa.stft(waveform)

display(Audio(waveform, rate=sample_rate))

plt.figure(figsize=(12, 5))
librosa.display.specshow(librosa.amplitude_to_db(np.abs(stft_spectrogram), ref=np.max), sr=sample_rate, x_axis='time', y_axis='linear')

音楽情報処理ersは、特にSTFTスペクトログラムの以下の性質に注意を払っています。


* **窓幅の決め方：** 窓幅が長いほど、周波数分解能が上がり、時間分解能が下がるので、タスクにとって丁度いい窓幅を見つける必要があります。
次に述べるように、STFTスペクトログラムは低周波数域の分解能不足が問題になりがちなので、「最低音域をちょうど区別できるくらい」の窓幅にするのがよくある基準です。

* **周波数域と人間の知覚的尺度のギャップ：** STFTスペクトログラムの周波数軸は線形になっており、隣り合う次元の中心周波数は一定間隔になっています。
対して人間が音高を知覚する尺度は対数尺度に近いため、「聴覚的に音高が等間隔な音が、STFTスペクトログラム上では等間隔にならない」というギャップが生じます。

* **周波数によってに異なる分解能：** FFTは全ての周波数を同じ窓幅で解析しているので、低周波数（＝長い波長）域では分解能が不足し、逆に高周波数（＝短い波長）域では分解能が過剰になります。
音楽信号内のどの音域も平等に扱いたい場合、音の高低によって分解能が異なるのは不便です。

STFTスペクトログラムは他の特徴量を求める第一歩にもなる、最も基本的な時間周波数表現の一つですが、
上記の性質もあって直接活用する場面は多くありません。
STFTは（隣接フレームがオーバーラップしていれば）可逆な変換なので、スペクトログラムに何かしらの処理を加えたあと、再び波形に戻したい時に使われることがあります（e.g. 音源分離）。

## Mel Spectrogram

STFTスペクトログラムの周波数軸を、「メル尺度」という音高の知覚的尺度に変換したスペクトログラムです。
STFT振幅スペクトログラムを、メルフィルタバンクを表す行列と乗算すると、メルスペクトログラムになります。

In [ ]:
mel_filterbank = librosa.filters.mel(
    sr=sample_rate, 
    fmin=30, 
    fmax=8000, 
    n_fft=2048, 
    n_mels=128,
)
mel_spectrogram = mel_filterbank @ np.abs(stft_spectrogram) # メルスペクトログラム＝メルフィルタバンクとSTFTスペクトログラムの積

plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
plt.imshow(mel_filterbank, aspect='auto', origin='lower', cmap='viridis')
plt.title('Mel filterbank')
plt.subplot(2, 1, 2)
librosa.display.specshow(librosa.power_to_db(mel_spectrogram, ref=np.max), sr=sample_rate, x_axis='time', y_axis='mel', fmin=30, fmax=8000)
plt.title('Mel spectrogram')

ディープラーニングが全部やってくれる昨今の研究では、どの分野も「**とりあえず入力はメルスぺにする**」が主流になっています。

`librosa`や`torchaudio`などのライブラリでは、`slaney`と`htk`という２つのメル尺度から選べます。

`slaney`は`MATLAB Auditory Toolbox`というMATLABライブラリに由来する尺度で、1kHz以下は線形、1kHz以上は対数スケールに変換しています。

$
mel_{htk}=
\begin{cases}
\frac{f-f_{min}}{66.67} & (f \leq 1000)\\
1000 + \frac{\log(f/1000)}{0.069} & (f \gt 1000)
\end{cases}
$

一方`htk`は、隠れマルコフモデルの実装に使うライブラリである`Hidden Markov Model Toolkit (HTK)`に由来する尺度で、こちらは全部対数スケールです。

$mel_{htk} = 2595.0 * \log_{10}(1.0 + f / 700.0)$

`librosa`は`slaney`、`torchaudio`は`htk`をデフォルトに設定しており、若干ややこしいです。

## CQT Spectrogram

## Chromagram

## MFCC

## Tempogram